<a href="https://colab.research.google.com/github/ShoAnn/indo-legal/blob/main/statutes_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# # Set up for running selenium in Google Colab
# ## You don't need to run this code if you do it in Jupyter notebook, or other local Python setting
# %%shell
# sudo apt -y update
# sudo apt install -y wget curl unzip
# wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
# dpkg -i libu2f-udev_1.1.4-1_all.deb
# wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
# dpkg -i google-chrome-stable_current_amd64.deb
# CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
# wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
# unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
# chmod +x /tmp/chromedriver
# mv /tmp/chromedriver /usr/local/bin/chromedriver

# pip install selenium chromedriver-autoinstaller requests beautifulsoup4

In [2]:




# #get element : "Pertanyaan"
# container = soup.find_all('div', class_='css-wnnb7z e1vjmfpm0')
# q_content = container[0].find('p')

# #get element : "Intisari jawaban" / short answer (sa)
# sa_container = soup.find('article', class_='css-1dk882h e9ek18z0')
# sa_content = sa_container.find('p')

# #get element : "Ulasan jawaban" / long answer (la)
# la_content = soup.find_all('div', string='Terima kasih atas pertanyaan Anda.').parent
# tobe_filtered = ['Terima kasih atas pertanyaan Anda', 'Artikel di bawah ini adalah pemutakhiran', 'Seluruh informasi hukum yang ada di Klinik hukumonline.com', 'Baca juga']
# la_content_filtered = [tag for tag in la_content if all(word not in tag.text.strip() for word in tobe_filtered)]

# select dasar hukum / legal basis (lb)
# Find all divs with id starting with 'ftn'
# lb = soup.find_all('div', id=lambda x: x.startswith('ftn') if x else None)
# lb_content = []
# for ftn_div in lb:
#     lb_content.extend(ftn_div.find_all('p'))



In [3]:
!pip install ydata_profiling

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.5/359.5 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 686.1/686.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 9.3 MB/s eta 0:00:00
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27080 sha256=362f2a16bee430e9097c18690c77f1ed7b26c5568458b059d84fe35efeb455b6
  Stored in directory: /root/.cache/pip/wheels/dd/91/29/a79cecb328d01739e64017b6fb9a1ab9d8cb1853098ec5966d
Successfully built htmlmin


In [4]:
import pandas as pd
import numpy as np
import re
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup
from ydata_profiling import ProfileReport


class Page:
    """
    Represents a webpage and provides methods to extract specific text elements.
    """

    def __init__(self, r_content):
        """
        Initializes the Page object with the HTML content.

        Args:
            r_content: The HTML content as bytes.
        """
        self.soup = BeautifulSoup(r_content, 'html.parser')
        self._question = None  # Initialize private variable for question
        self._sa = None  # Initialize private variable for short answer
        self._la = None  # Initialize private variable for long answer
        self._lb = None  # Initialize private variable for legal basis

    def _get_text_content(self, selector, tag_name='p'):
        """
        Extracts text content from a container identified by the selector (private method).

        Args:
            selector: A dictionary representing the selector criteria (e.g., class_, id).
            tag_name: The tag name to find within the container (default: 'p').

        Returns:
            The extracted text content (stripped), or None if not found.
        """
        container = self.soup.find(**selector)
        return container.find(tag_name).text.strip() if container else None

    def _get_filtered_text_content(self, selector=None, parent=None, tag_name='p', filter_words=None):
        """
        Extracts text content from a container, filtering out specific words (private method).

        Args:
            selector: A dictionary representing the selector criteria (e.g., class_, id). Defaults to None.
            parent: The parent element to search within (optional). Defaults to None.
            tag_name: The tag name to find within the container (default: 'p').
            filter_words: A list of words to exclude from the extracted content.

        Returns:
            A list of extracted text content (stripped) for each paragraph,
            excluding the filter words. Empty list if not found.
        """
        container = None
        if selector:
            container = self.soup.find(**selector)
        elif parent:
            container = parent  # Use the provided parent element

        if container:
            paragraphs = container.find_all(tag_name)
            return [tag.text.strip() for tag in paragraphs if all(word not in tag.text.strip() for word in filter_words)]
        return []

    def get_q(self):
        """
        Extracts and returns the question text from the page (cached).
        """
        if self._question is None:
            question_selector = {'class_': 'css-wnnb7z e1vjmfpm0'}
            self._question = self._get_text_content(question_selector)
        return self._question

    def get_sa(self):
        """
        Extracts and returns the short answer text from the page (cached).
        """
        if self._sa is None:
            sa_selector = {'class_': 'css-1dk882h e9ek18z0'}
            self._sa = self._get_text_content(sa_selector)
        return self._sa

    def get_la(self):
        """
        Extracts and returns the filtered long answer text from the page (cached).
        """
        if self._la is None:
            unique_tag = self.soup.find('p', string='Terima kasih atas pertanyaan Anda.')  # Find the "Thank you" paragraph
            if unique_tag:
                la_container = unique_tag.parent  # Get the parent container
                la_content = la_container.find_all('p')  # Find all paragraph elements within the container
                self._la = [p.text.strip() for p in la_content if all(word not in p.text.strip() for word in ['Terima kasih atas pertanyaan Anda', 'Artikel di bawah ini adalah pemutakhiran', 'Seluruh informasi hukum yang ada di Klinik hukumonline.com', 'Baca juga'])]  # Filter content and cache
            else:
                self._la = []  # Set empty list if not found
        return self._la


    def get_lb(self):
        """
        Extracts and returns a list of legal basis text snippets from the page (cached).
        """
        if self._lb is None:
            lb_selector = {'id': lambda x: x.startswith('ftn') if x else None}
            lb = self.soup.find_all('div', **lb_selector)  # Find all footnote divs

            lb_content = []
            for ftn_div in lb:
                paragraphs = ftn_div.find_all('p')  # Find all paragraphs within each footnote div
                if paragraphs:
                    lb_content.extend([p.text.strip() for p in paragraphs])  # Only extend if paragraphs found

            self._lb = lb_content
        return self._lb

def remove_duplicates(list2d):
  """
  Removes duplicate elements from a 2D list.

  Args:
      list2d: The 2D list to remove duplicates from.

  Returns:
      A new 2D list with duplicates removed.
  """

  # Convert the 2D list to a set to remove duplicates.
  unique_set = set(tuple(row) for row in list2d)

  # Convert the set back to a 2D list.
  unique_list = [list(row) for row in unique_set]

  return unique_list

In [5]:
#loop over pages
r = requests.get('https://www.hukumonline.com/klinik/pidana/page/')
list_lb = []

for page_num in tqdm(range(1, 31)):
    r = requests.get(f'https://www.hukumonline.com/klinik/pidana/page/{page_num}/')
    soup = BeautifulSoup(r.content, 'html.parser')
    posts = soup.find_all('div', class_='klinik-list')
    for post in posts:
        anchor = post.find('a')
        post_req = requests.get(f'https://www.hukumonline.com{anchor["href"]}')
        if post_req.status_code == 200:
            res = Page(post_req.content)

            #loop over lb's
            post_lb = res.get_lb()
            for lb in post_lb:
                list_lb.append([re.sub(r"\[(\d+)\]", "", lb)])
        else:
            print('request failed; status code: ', post_req.status_code)

100%|██████████| 30/30 [06:24<00:00, 12.82s/it]


In [13]:
import time


list_lb_set = remove_duplicates(list_lb)
df = pd.DataFrame(list_lb_set, columns=['peraturan'])
df.to_csv(f'peraturan_300page_{time.time()}.csv')

In [7]:
df

,peraturan
0,"Diajeng Wulan Christianti, Hukum Pidana Inter..."
1,Pasal 40 ayat (3) Perda DKI Jakarta 8/2007
2,Pasal 5 ayat (1) dan (2) Perkapolri 10/2010
3,Mahrus Ali. Pencemaran Nama Baik Melalui Sara...
4,Pasal 412 ayat 7 UU Penerbangan.
...,...
1272,Pasal 45A ayat (2) UU 1/2024
1273,Article 1 number 50 Government Regulation 22/...
1274,Pasal 24 ayat (1) UU TPKS
1275,"Adami Chazawi, Kejahatan Terhadap Harta Benda..."


In [8]:
# get all rows that starts with the string 'pasal' or 'Pasal'
pasal = df[df['peraturan'].str.contains('^ Pasal|^ pasal|^Pasal|^pasal', regex=True)]
pasal

,peraturan
1,Pasal 40 ayat (3) Perda DKI Jakarta 8/2007
2,Pasal 5 ayat (1) dan (2) Perkapolri 10/2010
4,Pasal 412 ayat 7 UU Penerbangan.
5,Pasal 26 ayat (1) dan (2) UU TPKS.
6,Pasal 67 ayat (1) UU TPKS
...,...
1266,Pasal 3 ayat (4) Permenhub 111/2015
1267,"Pasal 1 angka 5 Peraturan Menteri Pendidikan,..."
1271,Pasal 6 ayat (2) Perpol 5/2021
1272,Pasal 45A ayat (2) UU 1/2024


In [9]:
# 100 pages
    # before duplicates removal : 545 rows
    # after duplicates removal : 389 rows
    # starts with 'Pasal|pasal' : 261 rows
# 300 pages
    # before duplicates removal :  rows
    # after duplicates removal : 1229 rows
    # starts with 'Pasal|pasal' : 676 rows